<a href="https://colab.research.google.com/github/abhiram-gorla/UNET--Nuclei-Segmentation-for-Histopathology-images/blob/master/UNET_Nuclei_Segmentation_for_Histopathology_images_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>